In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from mongo_crud import mongo_crud





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "pat356"
db = mongo_crud("localhost", 53960, username, password, "AAC")
db.use("animals")


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(db.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'gs_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center([
        html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), href='/http://www.snhu.edu'),
        html.B(html.H1('SNHU CS-340 Dashboard by Patrick Pattison')),
        ]),
    
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter_type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset Filter', 'value': 'reset'}
            ],
            value='reset'
            
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_columns=[],
        selected_rows=[],
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',

                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    
    #  Water Rescue filter...
    if filter_type == 'water':
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Female'}],
                                        '$or' : [{'breed': 'Labrador Retriever Mix'},
                                                 {'breed': 'Chesa Bay Retr Mix'},
                                                 {'breed': 'Newfoundland Mix'},
                                                 {'breed': 'Newfoundland/Labrador Retriever'},
                                                 {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                 {'breed': 'Newfoundland/Great Pyrenees'}],
                                        '$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                 {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                       })))
    # Mountain Rescue filter...
    elif filter_type == 'mountain':
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'}],
                                        '$or' : [{'breed': 'German Shepard'},
                                                 {'breed': 'Alaskan Malamute'},
                                                 {'breed': 'Old English Sheepdog'},
                                                 {'breed': 'Rottweiler'},
                                                 {'breed': 'Siberian Husky'}],
                                        '$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                 {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                       })))
    # Disaster Rescue filter...
    elif filter_type == 'disaster':
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'}],
                                        '$or' : [{'breed': 'Doberman Pinscher'},
                                                 {'breed': 'German Shepard'},
                                                 {'breed': 'Golden Retriever'},
                                                 {'breed': 'Bloodhound'},
                                                 {'breed': 'Rottweiler'}],
                                        '$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                 {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                       })))
    # Reset filter...
    elif filter_type == 'reset':
        df = pd.DataFrame.from_records(db.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    #  Create a pie chart
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame = dff,
                values = values,
                names = names,
                color_discrete_sequence=px.colors.sequential.RdBu,
                width = 800,
                height = 500))]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0, 'location_lat'],dff.loc[0, 'location_long']], children=[
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app